In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from keras.models import load_model


# Load the dataset
data = pd.read_csv("train.csv")
data = data.iloc[:3000,:]
#data.drop(range(0,2000))
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [44]:
# Preprocess the data
# Convert categorical variables to numerical values
le = LabelEncoder()
data['Ship Mode'] = le.fit_transform(data['Ship Mode'])
data['Segment'] = le.fit_transform(data['Segment'])
data['Country'] = le.fit_transform(data['Country'])
data['Order Date'] = le.fit_transform(data['Order Date'])
data['Category'] = le.fit_transform(data['Category'])
# Select relevant columns for the model
X = data[[ 'Ship Mode', 'Segment','Country','Order Date','Category']].values
y = data['Sales'].values

# Split the data into training and testing sets
train_X = X[:int(X.shape[0] * 0.8)]
train_y = y[:int(y.shape[0] * 0.8)]

test_X = X[int(X.shape[0] * 0.8):]
test_y = y[int(y.shape[0] * 0.8):]

# Reshape the data for input into the LSTM network
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(32, input_shape=(1, 5)))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [45]:
# Train the LSTM model
model.fit(train_X, train_y, epochs=100, batch_size=1, verbose=1)
# Save the trained LSTM model
model.save("lstm_model.h5")



Epoch 1/100
2400/2400 [==============================] - 10s 3ms/step - loss: 295075.8438
Epoch 2/100
2400/2400 [==============================] - 7s 3ms/step - loss: 285731.5312
Epoch 3/100
2400/2400 [==============================] - 6s 3ms/step - loss: 278373.9062
Epoch 4/100
2400/2400 [==============================] - 6s 3ms/step - loss: 272462.4688
Epoch 5/100
2400/2400 [==============================] - 6s 3ms/step - loss: 267731.9375
Epoch 6/100
2400/2400 [==============================] - 7s 3ms/step - loss: 263967.8125
Epoch 7/100
2400/2400 [==============================] - 6s 3ms/step - loss: 260921.4375
Epoch 8/100
2400/2400 [==============================] - 7s 3ms/step - loss: 258544.1406
Epoch 9/100
2400/2400 [==============================] - 6s 3ms/step - loss: 256694.6875
Epoch 10/100
2400/2400 [==============================] - 7s 3ms/step - loss: 255267.2500
Epoch 11/100
2400/2400 [==============================] - 6s 3ms/step - loss: 254064.6406
Epoch 12/100
2400/

In [46]:
# Evaluate the LSTM model on the test data
test_loss = model.evaluate(test_X, test_y, verbose=1)
print("Test Loss: ", test_loss)

# Use the trained LSTM model to make predictions on the test data
predictions = model.predict(test_X)


19/19 [==============================] - 1s 2ms/step - loss: 1326630.8750
Test Loss:  1326630.875
19/19 [==============================] - 1s 2ms/step


In [47]:
# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(test_y, predictions))
print("RMSE:", rmse)

RMSE: 1151.7946861897378


In [48]:
from sklearn.metrics import r2_score

# Calculate the R^2 score
r2 = r2_score(test_y, predictions)
print("R^2:", r2)


R^2: -0.0042499777833582275


In [57]:
product_data = data['Product ID']  

In [58]:
# Preprocess the data
# Convert categorical variables to numerical values
le = LabelEncoder()
data['Product ID'] = le.fit_transform(data['Product ID'])

# Select relevant columns for the model
X = data[[ 'Product ID']].values
y = data['Sales'].values

# Split the data into training and testing sets
train_X = X[:int(X.shape[0] * 0.8)]
train_y = y[:int(y.shape[0] * 0.8)]

test_X = X[int(X.shape[0] * 0.8):]
test_y = y[int(y.shape[0] * 0.8):]

# Reshape the data for input into the LSTM network
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(32, input_shape=(1, 1)))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [59]:

# Use the trained LSTM model to make predictions on the test data
predictions = model.predict(test_X)

19/19 [==============================] - 1s 2ms/step


In [65]:

# Add the predictions as a new column in the product data DataFrame
product_data['prediction'] = predictions

product_data

0                                               FUR-BO-10001798
1                                               FUR-CH-10000454
2                                               OFF-LA-10000240
3                                               FUR-TA-10000577
4                                               OFF-ST-10000760
                                    ...                        
2996                                            OFF-EN-10001539
2997                                            FUR-FU-10000550
2998                                            TEC-AC-10003628
2999                                            TEC-MA-10004521
prediction    [[1.1865792], [1.1865792], [1.1865785], [0.156...
Name: Product ID, Length: 3001, dtype: object

In [67]:
# Sort the product data by the prediction column in descending order
sorted_product_data = product_data.sort_values('2')

# Display the top 10 products with the highest predictions
print(sorted_product_data.head(10))

<ipython-input-67-66d1eb1d3aa5>:2: FutureWarning: In a future version of pandas all arguments of Series.sort_values will be keyword-only
  sorted_product_data = product_data.sort_values('2')


ValueError: ignored